In [1]:
#Required Packages
import numpy as np
import pandas as pd
import re
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import HashingVectorizer
#from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [2]:
#Required Functions
def getWordsFromURL(url):
    return re.compile(r'[\:/?=\-&]+',re.UNICODE).split(url)
def purity_score(clusters, classes):
    """
    Calculate the purity score for the given cluster assignments and ground truth classes
    
    :param clusters: the cluster assignments array
    :type clusters: numpy.array
    
    :param classes: the ground truth classes
    :type classes: numpy.array
    
    :returns: the purity score
    :rtype: float
    """
    
    A = np.c_[(clusters,classes)]

    n_accurate = 0.

    for j in np.unique(A[:,0]):
        z = A[A[:,0] == j, 1]
        x = np.argmax(np.bincount(z))
        n_accurate += len(z[z == x])

    return n_accurate / A.shape[0]
#did not use below two funtions due to loading and computiong issues with html_data size
def pre_processing(html):
    soup=BeautifulSoup(html,"html.parser")
        # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    if soup:
        text1 = soup.get_text()
        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text1.splitlines())
        lines1= (line.replace('\n','') for line in lines)
        lines2= (line.replace('\t','') for line in lines)
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines2 for phrase in line.split("  "))
        # drop blank lines
        text = ' '.join(chunk for chunk in chunks if chunk)
        return clean_text(text)
    else:
        return 'No_data'
def clean_text(text):
    result = re.sub(r"http\S+", "", text)
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", result).split())
    #We're returning the tweet which is cleaned(after replacin' the above pattern to null which means deleting)

In [4]:
#loading, preprocessing of data sets
train=pd.read_csv('C:\\Users\\Ashok Chava\\Documents\\Kaggle\\AnalyticsVidhya\\train\\train.csv')
train.head()
train.shape

(53447, 4)

In [13]:
#loading, preprocessing of data sets
train['url_split']=train['Url'].apply(getWordsFromURL)
list1=train['url_split'].tolist()
list2=[i[2:] for i in list1]
list3=[' '.join(i) for i in list2]
train['vector']=list3
train.head()

,Webpage_id,Domain,Url,Tag,url_split,vector
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,4,"[http, www.fiercepharma.com, marketing, tecfid...",marketing tecfidera gilenya and aubagio s 3 wa...
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,4,"[http, www.fiercepharma.com, pharma, novo, equ...",pharma novo equipped to weather storm u s diab...
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,4,"[http, www.fiercepharma.com, pharma, another, ...",pharma another exec departs troubled endo and ...
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,4,"[http, www.fiercepharma.com, pharma, teva, buy...",pharma teva buy biosim specialist celltrion it...
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,4,"[http, www.fiercepharma.com, marketing, actres...",marketing actress marissa tomei partners aller...


In [12]:
#loading, preprocessing of data sets
test=pd.read_csv('C:\\Users\\Ashok Chava\\Documents\\Kaggle\\AnalyticsVidhya\\test_nvPHrOx.csv')
test['url_split']=test['Url'].apply(getWordsFromURL)
list1_t=test['url_split'].tolist()
list2_t=[i[2:] for i in list1_t]
list3_t=[' '.join(i) for i in list2_t]
test['vector']=list3_t
test.head()

,Webpage_id,Domain,Url,url_split,vector
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413,"[http, www.isrctn.com, ISRCTN57801413]",ISRCTN57801413
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,"[https, www.clinicaltrialsregister.eu, ctr, se...",ctr search trial 2006 006214 16 GB
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,"[https, www.clinicaltrialsregister.eu, ctr, se...",ctr search trial 2006 004265 34 LT
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,"[https, www.clinicaltrialsregister.eu, ctr, se...",ctr search trial 2010 022183 12 IT
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,"[https, www.clinicaltrialsregister.eu, ctr, se...",ctr search trial 2010 021349 36 NL


In [14]:
#loading, preprocessing of data sets
train['Tag']=train['Tag'].astype('category')
train['Tag']=train['Tag'].cat.codes
y_train=train['Tag'].values

In [15]:
#classification models
vectorizer = CountVectorizer()
x_train=vectorizer.fit_transform(train['vector'])

In [16]:
#classification models
clf = MultinomialNB(alpha=1,fit_prior=True,class_prior=None)
clf.fit(x_train, y_train)
x_train_p=clf.predict(x_train)
print(purity_score(x_train_p,y_train))

0.9064306696353397


In [18]:
x_test=vectorizer.transform(test['vector'])
x_test_p=clf.predict(x_test)
df_t = pd.DataFrame(x_test_p)
df_t.to_csv('15th_in.csv')

In [ ]:
#0 - clinicalTrials
#1 - conferences
#2 - forum
#3 - guidelines
#4 - news
#5 - others
#6 - profile
#7 - publication
#8 - thesis

In [ ]:
#Tested below models too
#clf=MLPClassifier(validation_fraction=0.2,verbose=True)
#clf.fit(x_train, y_train)